### Private Blob 내 csv 파일 processing

In [2]:
import pandas as pd

url = 'https://flightblobsample.blob.core.windows.net/container2/Lab01_flights.csv'
df = spark.createDataFrame(pd.read_csv(url, header = None))

In [3]:
import requests
import pandas as pd
from azure.storage.blob import BlockBlobService, BlobPermissions
from azure.storage.blob.baseblobservice import BaseBlobService
from datetime import datetime, timedelta

account_name = 'flightblobsample'
account_key = 'nwOuPQWCYnQEuep/EeR9dfsAJ/76NlO8cp3sZ1Bte5d33kGyG+TJfo6LJz5bjhPFxMqh498pmBZG6AEa35uIzg=='
container_name = 'container2'
filename = 'Lab01_flights.csv'

blob_service=BlockBlobService(account_name,account_key)
generator = blob_service.list_blobs(container_name)

for blob in generator:
  print(blob.name)

url = 'https://flightblobsample.blob.core.windows.net/container2/Lab01_flights.csv'
service = BaseBlobService(account_name=account_name, account_key=account_key)
token = service.generate_blob_shared_access_signature(container_name, filename, permission=BlobPermissions.READ, expiry=datetime.utcnow() + timedelta(hours=1),)
url_with_sas = url + '?' + token
print(url_with_sas)

df = spark.createDataFrame(pd.read_csv(url_with_sas, header = None))

In [4]:
flightDF = df.toDF('Month', 'DayofMonth', 'DayofWeek', 'DepTime', 'ArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum', 'ElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted')

flightDF.registerTempTable("flightTable")

####<분석 요건> 15분을 초과하는 지연 사례가 있었던 항공사
##### 1) sqlContext를 이용한 쿼리 수행

In [6]:
resultDF = sqlContext.sql(" SELECT UniqueCarrier, max(DepDelay) as maxDelayedTime \
                                    FROM ( \
                                           SELECT UniqueCarrier, DepDelay \
                                          FROM flightTable \
                                          WHERE DepDelay > 15 \
                                    ) d \
                                    GROUP BY UniqueCarrier \
                                    ORDER BY maxDelayedTime DESC ")
resultDF.collect()

##### 2) sql magic을 사용한 쿼리 수행

In [8]:
%sql
SELECT UniqueCarrier, max(DepDelay) as maxDelayedTime 
FROM ( 
      SELECT UniqueCarrier, DepDelay
      FROM flightTable
      WHERE DepDelay > 15
) d
GROUP BY UniqueCarrier
ORDER BY maxDelayedTime DESC